In [88]:
import os, sys
import glob
import time
from datetime import timedelta, datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
from gezi import tqdm
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
#setting offilne
plotly.offline.init_notebook_mode(connected=True)

In [4]:
d_click = pd.read_csv('../input/train/click_log.csv')
d_click.head()

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1


In [7]:
for col in d_click.columns:
  print(col, d_click[col].nunique())

time 91
user_id 900000
creative_id 2481135
click_times 41


In [5]:
d_ad = pd.read_csv('../input/train/ad.csv')
d_ad.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202
2,7,7,\N,5,148,297
3,8,8,\N,5,713,213
4,9,9,\N,5,695,213


In [8]:
d_ad = d_ad.replace('\\N', 0)

In [10]:
d_ad.product_id = d_ad.product_id.astype(np.int32)
d_ad.industry = d_ad.industry.astype(np.int32)

In [12]:
d_ad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2481135 entries, 0 to 2481134
Data columns (total 6 columns):
creative_id         int64
ad_id               int64
product_id          int32
product_category    int64
advertiser_id       int64
industry            int32
dtypes: int32(2), int64(4)
memory usage: 94.6 MB


In [13]:
d = d_click.merge(d_ad, how="left", on="creative_id")

In [14]:
d.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,504423,30673,3,32638,319
1,65,30920,3072255,1,2642300,1261,2,6783,6
2,56,30920,2361327,1,2035918,1261,2,6783,6
3,6,309204,325532,1,292523,27081,3,32066,242
4,59,309204,2746730,1,2362208,0,18,14682,88


In [15]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30082771 entries, 0 to 30082770
Data columns (total 9 columns):
time                int64
user_id             int64
creative_id         int64
click_times         int64
ad_id               int64
product_id          int32
product_category    int64
advertiser_id       int64
industry            int32
dtypes: int32(2), int64(7)
memory usage: 2.0 GB


In [16]:
d.describe()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
count,3.008277e+07,3.008277e+07,3.008277e+07,3.008277e+07,3.008277e+07,3.008277e+07,3.008277e+07,3.008277e+07,3.008277e+07
mean,4.801738e+01,4.498556e+05,1.676532e+06,1.063127e+00,1.447917e+06,3.928835e+03,8.128011e+00,2.119445e+04,1.610350e+02
std,2.592504e+01,2.599939e+05,1.308054e+06,2.982831e-01,1.120569e+06,9.123197e+03,7.055121e+00,1.406527e+04,1.290624e+02
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00
25%,2.600000e+01,2.246130e+05,4.158100e+05,1.000000e+00,3.711100e+05,0.000000e+00,2.000000e+00,1.098600e+04,8.000000e+00
50%,4.900000e+01,4.498040e+05,1.508864e+06,1.000000e+00,1.310317e+06,1.290000e+02,3.000000e+00,1.810300e+04,2.020000e+02
75%,7.100000e+01,6.752420e+05,2.740464e+06,1.000000e+00,2.355111e+06,1.873000e+03,1.800000e+01,3.003400e+04,2.930000e+02
max,9.100000e+01,9.000000e+05,4.445718e+06,1.520000e+02,3.812200e+06,4.431300e+04,1.800000e+01,6.296500e+04,3.350000e+02


In [17]:
for col in d.columns:
  print(col, d[col].nunique())

time 91
user_id 900000
creative_id 2481135
click_times 41
ad_id 2264190
product_id 33273
product_category 18
advertiser_id 52090
industry 326


In [19]:
user=d.groupby(["time","user_id"])['click_times'].count().reset_index(name="count")
user

,time,user_id,count
0,1,12,1
1,1,22,3
2,1,25,1
3,1,33,2
4,1,34,1
...,...,...,...
19825286,91,899987,2
19825287,91,899988,2
19825288,91,899991,1
19825289,91,899998,1


In [21]:
user['count'].max()

401

In [29]:
cids = d.groupby('creative_id')['creative_id'].count().reset_index(name="count")

creative_id
1          5
4          6
7          4
8          1
9          1
          ..
4445714    1
4445715    3
4445716    1
4445717    1
4445718    2
Length: 2481135, dtype: int64

In [38]:
cids['count'].mean()

12.124600636402292

In [39]:
cids['count'].max()

103033

In [41]:
d[d.creative_id==2746730]

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
4,59,309204,2746730,1,2362208,0,18,14682,88
146033,68,317233,2746730,1,2362208,0,18,14682,88
2874484,69,464429,2746730,1,2362208,0,18,14682,88
8128837,67,74772,2746730,1,2362208,0,18,14682,88
8540266,67,770030,2746730,1,2362208,0,18,14682,88
13924336,58,251038,2746730,1,2362208,0,18,14682,88
14854757,68,30122,2746730,1,2362208,0,18,14682,88
15156717,59,317331,2746730,1,2362208,0,18,14682,88
15333584,66,327020,2746730,1,2362208,0,18,14682,88
15600761,66,341151,2746730,1,2362208,0,18,14682,88


In [42]:
d[d.ad_id==2362208]

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
4,59,309204,2746730,1,2362208,0,18,14682,88
146033,68,317233,2746730,1,2362208,0,18,14682,88
2874484,69,464429,2746730,1,2362208,0,18,14682,88
8128837,67,74772,2746730,1,2362208,0,18,14682,88
8540266,67,770030,2746730,1,2362208,0,18,14682,88
13924336,58,251038,2746730,1,2362208,0,18,14682,88
14854757,68,30122,2746730,1,2362208,0,18,14682,88
15156717,59,317331,2746730,1,2362208,0,18,14682,88
15333584,66,327020,2746730,1,2362208,0,18,14682,88
15600761,66,341151,2746730,1,2362208,0,18,14682,88


In [43]:
len(cids)

2481135

In [47]:
len(cids[cids['count'] < 5]) / len(cids)

0.7591175006599802

In [49]:
d['mark'] = 'train'

In [50]:
d

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,mark
0,9,30920,567330,1,504423,30673,3,32638,319,train
1,65,30920,3072255,1,2642300,1261,2,6783,6,train
2,56,30920,2361327,1,2035918,1261,2,6783,6,train
3,6,309204,325532,1,292523,27081,3,32066,242,train
4,59,309204,2746730,1,2362208,0,18,14682,88,train
...,...,...,...,...,...,...,...,...,...,...
30082766,6,30920,228129,1,204785,0,18,14681,297,train
30082767,57,30920,2361397,1,2035987,1261,2,6783,6,train
30082768,26,30920,629802,1,559183,0,18,14678,26,train
30082769,86,30920,2713031,1,2333817,0,18,32019,25,train


In [51]:
d_click2 = pd.read_csv('../input/test/click_log.csv')
d_ad2 = pd.read_csv('../input/test/ad.csv')
d_ad2 = d_ad2.replace('\\N', 0)
d_ad2.product_id = d_ad2.product_id.astype(np.int32)
d_ad2.industry = d_ad2.industry.astype(np.int32)
d2 = d_click2.merge(d_ad2, how="left", on="creative_id")
d2['mark'] = 'test'

In [53]:
d2

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,mark
0,20,3131989,645764,1,573314,58,2,14689,6,test
1,20,3131989,1027422,1,902764,129,2,42272,6,test
2,20,3131989,1106443,1,970829,2171,2,37513,322,test
3,20,3131989,629802,1,559183,0,18,14678,26,test
4,59,3131989,2839769,1,2441288,129,2,35328,6,test
...,...,...,...,...,...,...,...,...,...,...
33585507,16,3131989,585914,1,521003,129,2,38382,6,test
33585508,13,3131989,749134,1,662371,2171,2,37513,322,test
33585509,12,3131989,589027,1,523554,0,18,29595,26,test
33585510,61,3131989,2404831,1,2072705,111,2,17284,242,test


In [54]:
d = pd.concat([d, d2])

In [55]:
d

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,mark
0,9,30920,567330,1,504423,30673,3,32638,319,train
1,65,30920,3072255,1,2642300,1261,2,6783,6,train
2,56,30920,2361327,1,2035918,1261,2,6783,6,train
3,6,309204,325532,1,292523,27081,3,32066,242,train
4,59,309204,2746730,1,2362208,0,18,14682,88,train
...,...,...,...,...,...,...,...,...,...,...
33585507,16,3131989,585914,1,521003,129,2,38382,6,test
33585508,13,3131989,749134,1,662371,2171,2,37513,322,test
33585509,12,3131989,589027,1,523554,0,18,29595,26,test
33585510,61,3131989,2404831,1,2072705,111,2,17284,242,test


In [56]:
cids = d.groupby('creative_id')['creative_id'].count().reset_index(name="count")

In [57]:
aids = d.groupby('ad_id')['ad_id'].count().reset_index(name="count")

In [58]:
cids

,creative_id,count
0,1,11
1,3,1
2,4,12
3,5,1
4,6,1
...,...,...
3412767,4445716,1
3412768,4445717,1
3412769,4445718,3
3412770,4445719,2


In [59]:
aids

,ad_id,count
0,1,11
1,3,1
2,4,12
3,5,1
4,6,1
...,...,...
3027355,3812198,1
3027356,3812199,1
3027357,3812200,3
3027358,3812201,2


In [64]:
len(aids[aids['count'] > 5]) / len(aids)

0.2892546641298029

In [65]:
aids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3027360 entries, 0 to 3027359
Data columns (total 2 columns):
ad_id    int64
count    int64
dtypes: int64(2)
memory usage: 46.2 MB


In [75]:
aids

,ad_id,count
0,1,11
1,3,1
2,4,12
3,5,1
4,6,1
...,...,...
3027355,3812198,1
3027356,3812199,1
3027357,3812200,3
3027358,3812201,2


In [79]:
len(aids[aids['count'] > 5])

875678

In [77]:
len(aids)

3027360

In [80]:
import gezi

In [93]:
wc = gezi.WordCounter()

In [94]:
for i in tqdm(range(len(aids))):
  row = aids.iloc[i]
  wc.add(str(row['ad_id']), row['count'])

100%|██████████| 3027360/3027360 [11:30<00:00, 4384.25it/s]


In [95]:
wc.save('../input/all/ad_ids.vocab')

total_count 63668283 unknown_count 0 total_word 3027360


In [96]:
wc = gezi.WordCounter()

In [97]:
for i in tqdm(range(len(cids))):
  row = cids.iloc[i]
  wc.add(str(row['creative_id']), row['count'])

100%|██████████| 3412772/3412772 [14:33<00:00, 3906.45it/s]


In [98]:
wc.save('../input/all/creative_ids.vocab')

total_count 63668283 unknown_count 0 total_word 3412772


In [99]:
m = d.user_id % 10 == 0
d[m]

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,mark
0,9,30920,567330,1,504423,30673,3,32638,319,train
1,65,30920,3072255,1,2642300,1261,2,6783,6,train
2,56,30920,2361327,1,2035918,1261,2,6783,6,train
196,75,309220,3248730,1,2794783,1849,2,57780,317,train
197,60,309220,2805987,1,2412322,0,5,15855,84,train
...,...,...,...,...,...,...,...,...,...,...
33584703,71,3131940,3401237,1,2927022,0,5,17189,73,test
33584704,1,3131940,35391,1,34286,129,2,22123,6,test
33584705,86,3131940,4143277,1,3554534,0,5,14398,202,test
33584706,42,3131940,46381,1,43443,145,2,27571,60,test


In [ ]:
vin